# sPyTial Dataclass Input Builder Demo

This notebook demonstrates the dataclass input builder functionality in sPyTial. The input builder allows you to:

1. Define a dataclass with spatial annotations
2. Generate an interactive HTML interface for building instances of that dataclass
3. Collect spatial constraint specifications from annotated dataclasses

The input builder is the "input analog" to `spytial.diagram()` - while `diagram()` visualizes existing data, the input builder creates interfaces for building new data structures.

In [ ]:
# First, ensure the package is installed in development mode
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import spytial
from dataclasses import dataclass, field
from typing import List, Optional

## Basic Dataclass Definition

Let's start with a simple dataclass representing a person with spatial annotations:

In [ ]:
@dataclass
@spytial.orientation(selector='name', directions=['above'])
@spytial.atomColor(selector='name', value='blue')
class Person:
    name: str = ""
    age: int = 0
    email: str = ""
    
print(f"Defined Person dataclass: {Person}")
print(f"Fields: {[f.name for f in Person.__dataclass_fields__.values()]}")

## Generate Input Builder Interface

Now we can use the `spytial.build_input()` function to create an interactive interface for building Person instances:

In [ ]:
# Generate input builder HTML file
html_file = spytial.build_input(
    Person, 
    title="Person Input Builder"
)

print(f"Generated input builder: {html_file}")
print(f"File exists: {os.path.exists(html_file)}")



## Alternative API: input_builder()

You can also use the convenience function `spytial.input_builder()` which mirrors the `spytial.diagram()` API:

In [ ]:
# Using the convenience function
html_file2 = spytial.input_builder(Person, auto_open=False)
print(f"Generated with input_builder(): {html_file2}")

## Complex Dataclass with Nested Structures

Let's create a more complex example with nested dataclasses and relationships:

In [ ]:
@dataclass
@spytial.atomColor(selector='street', value='green')
class Address:
    street: str = ""
    city: str = ""
    state: str = ""
    zip_code: str = ""

@dataclass
@spytial.orientation(selector='title', directions=['above'])
@spytial.group(field='skills', groupOn=0, addToGroup=1)
class Employee:
    name: str = ""
    title: str = ""
    department: str = ""
    skills: List[str] = field(default_factory=list)
    address: Optional[Address] = None
    salary: float = 0.0
    
print(f"Defined Employee dataclass with nested Address")
print(f"Employee fields: {list(Employee.__dataclass_fields__.keys())}")
print(f"Address fields: {list(Address.__dataclass_fields__.keys())}")

In [ ]:
# Generate input builder for the complex Employee dataclass
employee_html = spytial.build_input(
    Employee,
    method='file',
    auto_open=False,
    title="Employee Input Builder - Complex Example"
)

print(f"Generated complex input builder: {employee_html}")

## Inspecting Generated CnD Specs

We can also look at the generated CnD specifications to understand how the annotations are being processed:

In [ ]:
# Import the internal functions to inspect the spec generation
from spytial.dataclassbuilder import collect_dataclass_annotations, generate_cnd_spec

# Collect annotations for Person
person_annotations = collect_dataclass_annotations(Person)
print("Person Annotations:")
print(person_annotations)

print("\n" + "="*50 + "\n")

# Generate CnD spec for Employee (with nested Address)
employee_spec = generate_cnd_spec(Employee)
print("Employee CnD Spec (YAML):")
print(employee_spec)

## Tree Structure Example

Let's create a tree node dataclass to show how recursive structures work with the input builder:

In [ ]:
from typing import List, Optional

@dataclass
@spytial.orientation(selector='value', directions=['above'])
@spytial.group(field='children', groupOn=0, addToGroup=1)
@spytial.atomColor(selector='value', value='red')
class TreeNode:
    value: str = ""
    children: List['TreeNode'] = field(default_factory=list)
    parent: Optional['TreeNode'] = None
    
    def __str__(self):
        return f"TreeNode(value='{self.value}', children={len(self.children)})"

print(f"Defined TreeNode dataclass")
print(f"TreeNode fields: {list(TreeNode.__dataclass_fields__.keys())}")

In [ ]:
# Generate input builder for TreeNode
tree_html = spytial.build_input(
    TreeNode,
    method='file',
    auto_open=False,
    title="Tree Node Input Builder"
)

print(f"Generated tree input builder: {tree_html}")

# Inspect the tree annotations
tree_annotations = collect_dataclass_annotations(TreeNode)
print("\nTreeNode Annotations:")
print(tree_annotations)

## Error Handling Examples

Let's demonstrate error handling for classes that aren't dataclasses:

In [ ]:
# Regular class (not a dataclass)
class RegularClass:
    def __init__(self, name):
        self.name = name

# This should raise an error
try:
    spytial.build_input(RegularClass)
except ValueError as e:
    print(f"Expected error: {e}")

# Test with invalid method
try:
    spytial.build_input(Person, method='invalid')
except ValueError as e:
    print(f"Expected error: {e}")

## Inline HTML Generation

You can also generate the HTML content directly as a string instead of saving to a file:

In [ ]:
# Generate HTML content as string
html_content = spytial.build_input(Person, method='inline')

print(f"Generated HTML content length: {len(html_content)} characters")
print(f"Contains 'structured-input-graph': {'structured-input-graph' in html_content}")
print(f"Contains CnD Core script: {'cnd-core' in html_content}")

# Show first 500 characters
print("\nFirst 500 characters:")
print(html_content[:500] + "...")

## Summary

The sPyTial dataclass input builder provides:

1. **`spytial.build_input(dataclass_type, ...)`** - Core function for generating input interfaces
2. **`spytial.input_builder(dataclass_type, ...)`** - Convenience function mirroring `spytial.diagram()` API
3. **Automatic annotation collection** - Recursively processes spatial annotations from dataclasses
4. **Nested dataclass support** - Handles complex structures with nested dataclasses
5. **CnD spec generation** - Converts annotations to YAML format for the input interface
6. **Interactive HTML interface** - Uses the Cope and Drag input components for data building

### Key Features:
- Works with any `@dataclass` decorated class
- Automatically identifies which dataclass is being built through type analysis
- Supports all sPyTial spatial annotations (`@orientation`, `@group`, `@atomColor`, etc.)
- Handles recursive and self-referential dataclass structures
- Provides both file output and inline HTML generation
- Integrates with existing sPyTial annotation system

### API Design:
```python
# Primary API
spytial.build_input(dataclass_type, method='file', auto_open=True, title=None)

# Convenience API (mirrors spytial.diagram)
spytial.input_builder(dataclass_type, **kwargs)
```

The input builder seamlessly integrates with sPyTial's existing annotation system and provides a natural "input analog" to the visualization functionality.